In [11]:
import warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
import sys

In [12]:
partition = 300

In [13]:
!{sys.executable} -m pip install pytorch-tabnet --upgrade


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [14]:
trainpath = f'../../../../data/top30groups/OneHotLongLatCombined/train1/train{partition}.csv'
testpath = f'../../../../data/top30groups/OneHotLongLatCombined/test1/test{partition}.csv'

traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

In [15]:
import torch
print(torch.cuda.is_available())

True


In [16]:
import torch
torch.cuda.empty_cache()


In [17]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

def split_data(dftrain, dftest):
    Xtrain = dftrain.drop(columns=['gname']).values.astype(float)
    Ytrain = dftrain['gname'].values
    Xtest = dftest.drop(columns=['gname']).values.astype(float)
    Ytest = dftest['gname'].values

    le = LabelEncoder()
    Ytrain = le.fit_transform(Ytrain)
    Ytest = le.transform(Ytest)

    #y_pred_decoded = model.label_encoder.inverse_transform(y_pred)
    y_true_decoded = le.inverse_transform(Ytest)

    return Xtrain, Ytrain, Xtest, Ytest, y_true_decoded, le



In [18]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier

class TabNetClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, lambda_sparse=1e-3, optimizer_params=None):
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.lambda_sparse = lambda_sparse
        self.optimizer_params = optimizer_params or {'lr': 0.01}
        self.model = None

    def fit(self, X, y):
        self.model = TabNetClassifier(
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            lambda_sparse=self.lambda_sparse,
            optimizer_params=self.optimizer_params,
            seed=42,
            verbose=0
        )

        self.model.fit(
            X, y,
            eval_set=[(X, y)],
            max_epochs=200,
            patience=20,
            batch_size=1024,
            virtual_batch_size=128,
            eval_metric=['accuracy']
        )

        self.classes_ = np.unique(y)  # ✅ Needed for sklearn compatibility
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        preds = self.predict(X)
        return (preds == y).mean()


In [19]:
#from pytorch_tabnet.sklearn import TabNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

def find_best_tabnet(Xtrain, Ytrain, n_iter=20):
    print("Starting TabNet grid search")
    print("CUDA available:", torch.cuda.is_available())

    param_dist = {
        'n_d': [8, 16, 24],
        'n_a': [8, 16, 24],
        'n_steps': [3, 4, 5],
        'gamma': [1.0, 1.3, 1.5],
        'lambda_sparse': [1e-4, 1e-3, 1e-2],
        'optimizer_params': [{'lr': 0.01}]
    }

    random_search = RandomizedSearchCV(
        estimator=TabNetClassifierWrapper(),
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(Xtrain, Ytrain)
    print("Best parameters:", random_search.best_params_)
    print("Best accuracy:", random_search.best_score_)

    return random_search.best_params_



In [20]:
Xtrain, Ytrain, Xtest, Ytest, Ytest_decoded, le = split_data(traindata, testdata)
best_tabnet_params = find_best_tabnet(Xtrain, Ytrain)


# Re-initialize TabNet with best params
final_model = TabNetClassifier(
    **{k: v for k, v in best_tabnet_params.items()},
    verbose=1,
    seed=42
)

# Retrain on full training data
final_model.fit(
    Xtrain, Ytrain,
    max_epochs=100,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128
)



Starting TabNet grid search
CUDA available: True
Fitting 5 folds for each of 20 candidates, totalling 100 fits

Early stopping occurred at epoch 21 with best_epoch = 1 and best_val_0_accuracy = 0.04345


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 22 with best_epoch = 2 and best_val_0_accuracy = 0.04762


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 200 with best_epoch = 182 and best_val_0_accuracy = 0.95813


...


Early stopping occurred at epoch 196 with best_epoch = 176 and best_val_0_accuracy = 0.94603


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 200 with best_epoch = 188 and best_val_0_accuracy = 0.94127


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 153 with best_epoch = 133 and best_val_0_accuracy = 0.88683
Best parameters: {'optimizer_params': {'lr': 0.01}, 'n_steps': 5, 'n_d': 24, 'n_a': 16, 'lambda_sparse': 0.01, 'gamma': 1.3}
Best accuracy: 0.5593650793650793
epoch 0  | loss: 4.87162 |  0:00:00s
epoch 1  | loss: 4.3678  |  0:00:01s
epoch 2  | loss: 4.07594 |  0:00:01s
epoch 3  | loss: 3.89325 |  0:00:02s
epoch 4  | loss: 3.77504 |  0:00:02s
epoch 5  | loss: 3.67213 |  0:00:03s
epoch 6  | loss: 3.57884 |  0:00:04s
epoch 7  | loss: 3.5511  |  0:00:04s
epoch 8  | loss: 3.52586 |  0:00:05s
epoch 9  | loss: 3.47894 |  0:00:05s
epoch 10 | loss: 3.42511 |  0:00:06s
epoch 11 | loss: 3.37657 |  0:00:07s
epoch 12 | loss: 3.34968 |  0:00:07s
epoch 13 | loss: 3.31838 |  0:00:08s
epoch 14 | loss: 3.28    |  0:00:08s
epoch 15 | loss: 3.23304 |  0:00:09s
epoch 16 | loss: 3.17275 |  0:00:10s
epoch 17 | loss: 3.12718 |  0:00:10s
epoch 18 | loss: 3.13311 |  0:00:11s
epoch 19 | loss: 3.10199 |  0:00:11s
epoch 2

In [21]:
_, _, _, _, _, le = split_data(traindata, testdata)


In [22]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

file_path = os.path.join("results", f"gtd{partition}.txt")

# Predict class indices for test set
y_pred = final_model.predict(Xtest)
y_pred_decoded = le.inverse_transform(y_pred)
y_true_decoded = le.inverse_transform(Ytest)

# Make sure the directory exists
os.makedirs("results", exist_ok=True)

# Compute accuracy from decoded labels
acc = accuracy_score(y_true_decoded, y_pred_decoded)

# Write metrics to file
with open(file_path, "w") as file:
    file.write(f"Accuracy: {acc:.4f}\n")
    file.write(f"Precision: {precision_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"Recall: {recall_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"F1 Score: {f1_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")


In [23]:
print(classification_report(Ytest_decoded, y_pred_decoded))

                                                  precision    recall  f1-score   support

                          Abu Sayyaf Group (ASG)       0.60      0.43      0.50        90
        African National Congress (South Africa)       0.69      0.63      0.66        90
                                Al-Qaida in Iraq       0.66      0.76      0.70        90
        Al-Qaida in the Arabian Peninsula (AQAP)       0.57      0.62      0.60        90
                                      Al-Shabaab       0.83      0.64      0.72        90
             Basque Fatherland and Freedom (ETA)       0.79      0.71      0.75        90
                                      Boko Haram       0.61      0.44      0.51        90
  Communist Party of India - Maoist (CPI-Maoist)       0.18      0.30      0.22        90
       Corsican National Liberation Front (FLNC)       0.89      0.82      0.86        90
                       Donetsk People's Republic       0.79      0.76      0.77        90
Farabundo

In [24]:
def plot_confusion_matrix(y_true, y_pred, labels):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"results/confusion_matrix_partition_{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [25]:
# Get all unique class labels from the truths
class_labels = np.unique(Ytest_decoded)

plot_confusion_matrix(Ytest_decoded, y_pred_decoded, labels=class_labels)



Saved confusion matrix for partition 300 to results/confusion_matrix_partition_300.png
